In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering  as AC
from sklearn.manifold import MDS

def matrix_from_sparse(data, diag=0, offdiag=1):
    """Construct a (dis)similarity matrix from a sparse representation.
    """
    # Un-sparsify
    dmatrix = data.unstack()
    # Fill out the axes
    orfs = sorted(list(set(dmatrix.index) | set(dmatrix.columns)))
    dmatrix = dmatrix.reindex(index=orfs, columns=orfs)
    # Mirror the data over the diagonal
    dmatrix = dmatrix.fillna(dmatrix.T)
    # Set the diagonal
    np.fill_diagonal(dmatrix.values, 0)
    # Fill any other gaps
    dmatrix = dmatrix.fillna(1)
    return dmatrix

In [ ]:
data = pd.read_table('data/core.a.mags.muri.g.contigs.pilon.ctrim-50.dbCAN-hits.blastp_diss.tsv',
                     names=['qseqid', 'sseqid', 'dissimilarity'], index_col=['qseqid', 'sseqid'])
dmatrix = matrix_from_sparse(data.dissimilarity)

In [ ]:
domains = pd.read_table('data/core.a.mags.muri.g.contigs.pilon.ctrim-50.dbCAN-hits.domains.tsv',
                     names=['domain_id', 'domain_',
                            'query_id', 'query_length',
                            'evalue', 'domain_start',
                            'domain_end', 'query_start',
                            'query_end', 'score'])

In [ ]:
opu = pd.read_table('data/core.a.mags.muri.g.contigs.pilon.ctrim-50.dbCAN-hits.annot_table.tsv')
opu[opu.denovo_clust == 'Opu0584']

In [ ]:
ii = opu[((opu.cog == 'COG0366') | (opu.denovo_clust == 'Opu0584') | (opu.domain_structure == 'GH13')) &
             (opu.locus_tag.isin(dmatrix.index))].locus_tag.unique()

In [ ]:
ac = AC(n_clusters=1, affinity='precomputed', linkage='average').fit(dmatrix.loc[ii, ii])
cluster = pd.DataFrame({'cluster': ac.labels_}, index=dmatrix.loc[ii, ii].index)

In [ ]:
from scipy.cluster.hierarchy import dendrogram

children = ac.children_

distance = np.arange(children.shape[0])
position = np.arange(2, children.shape[0]+2)

linkage_matrix = np.column_stack([
    children, distance, position]
).astype(float)

fig, ax = plt.subplots(figsize=(10, 15))
ax.set_frame_on(False)

dgrm = dendrogram(linkage_matrix, orientation='left', ax=ax, labels=cluster.index)

leaf_order = pd.Series(dgrm['ivl'])
cluster_start, cluster_stop = list(leaf_order[leaf_order.isin(opu[opu.denovo_clust == 'Opu0584'].locus_tag)].iloc[[0,-1]].index)
ax.vlines(0, (cluster_start + 1)*10, (cluster_stop + 1)*10)

fig.tight_layout()
fig.savefig('test.pdf')

In [ ]:
import matplotlib as mpl

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(1 - dmatrix.loc[leaf_order, leaf_order])


rect = mpl.patches.Rectangle((cluster_start - 1, cluster_start - 1),
                             (cluster_stop - cluster_start) + 2,
                             (cluster_stop - cluster_start) + 2,
                             facecolor='none', lw=1, edgecolor='r')
ax.add_patch(rect)

#ax.vlines(cluster_start , cluster_start, cluster_stop, color='r')
#ax.vlines(cluster_stop, cluster_start, cluster_stop, color='r')
#ax.hlines(cluster_start, cluster_start, cluster_stop, color='r')
#ax.hlines(cluster_stop, cluster_start, cluster_stop, color='r')

TODO: Check out OTU-7.vA.a.scaffolds.pilon.ctrim_01769 , and OTU-1.vB.a.scaffolds.pilon.ctrim_01382 ,
as well as OTU-7.vA.a.scaffolds.pilon.ctrim_02383 , and OTU-1.vB.a.scaffolds.pilon.ctrim_00001
each pair of which have perfect identity despite being in different backgrounds.

In [ ]:
opu = pd.read_table('res/core.a.mags.muri.dbCAN-hits.denovo-clust.tsv', names=['query_id', 'cluster'])
opu[opu.cluster == 'Opu0819']